#### Задание
Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
1. Проверить насколько хорошо она суммаризирует

In [37]:
import pandas as pd
import numpy as np

In [1]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [2]:
import json
import random

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [3]:
train_records = read_gazeta_records("gazeta_train.txt")
val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

In [8]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [50]:
import razdel

def calc_lead_n_score(records, summary_col = 'summary', n=3, lower=True, nrows=1000):
    references = []
    predictions = []

    for i, record in enumerate(records):
        if i >= nrows:
            break

        summary = record[summary_col]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        text = text if not lower else text.lower()
        sentences = [sentence.text for sentence in razdel.sentenize(text)]
        prediction = " ".join(sentences[:n])
        predictions.append(prediction)

    calc_scores(references, predictions)

calc_lead_n_score(test_records, n=1)

Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: православный телеканал «спас», учредителем которого является московская патриархия, запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер в тверской области.
BLEU:  0.19177311186434495
ROUGE:  {'rouge-1': {'r': 0.37762764047433917, 'p': 0.22208274285774904, 'f': 0.23804097238957525}, 'rouge-2': {'r': 0.15833772153385062, 'p': 0.09647636782929753, 'f': 0.10027796832321115}, 'rouge-l': {'r': 0.34937017731940756, 'p': 0.2022959168891477, 'f': 0.21799992093276083}}


In [31]:
import torch

In [22]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'cointegrated/rut5-base-absum'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
# model.cuda();
model.eval();

def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0, 
    **kwargs
):
    """
    Summarize the text
    The following parameters are mutually exclusive:
    - n_words (int) is an approximate number of words to generate.
    - compression (float) is an approximate length ratio of summary and original text.
    """
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **x, 
            max_length=max_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty, 
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [32]:
sum_pred = summarize(test_records[0]['text'], len(test_records[0]['summary'].split()))

In [33]:
sum_pred

'Американское аэрокосмическое агентство NASA объявило названия четырех космических миссий, которые в скором времени могут быть выбраны для реализации.'

In [34]:
test_records[0]['summary']

'В NASA назвали четыре миссии в дальний космос, которые в этом десятилетии могут быть запущены американцами. Среди них — две миссии по изучению Венеры, полет к спутнику Юпитера и экспедиция к Тритону, спутнику Нептуна.'

In [38]:
df_test = pd.DataFrame(test_records)

In [46]:
df_m = df_test[:10]

In [47]:
df_m['summary_pred'] = df_m.apply(lambda x: summarize(x['text'], len(x['summary'].split())), axis=1)

<ipython-input-47-d263e440c7d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m['summary_pred'] = df_m.apply(lambda x: summarize(x['text'], len(x['summary'].split())), axis=1)


In [48]:
df_m

,url,text,title,summary,date,summary_pred
0,https://www.gazeta.ru/science/2020/02/14_a_129...,Американское аэрокосмическое агентство NASA ог...,"Венера, Ио или Тритон: куда полетит NASA","В NASA назвали четыре миссии в дальний космос,...",2020-02-14 16:39:11,Американское аэрокосмическое агентство NASA об...
1,https://www.gazeta.ru/social/2020/02/28/129806...,Около 11 тысяч зрителей увидели все самое лучш...,«Люди в Бурятии очень талантливые»,25 и 26 февраля в Кремлевском дворце съездов п...,2020-02-28 10:44:13,В Бурятии прошел праздничный концерт «Танцуют ...
2,https://www.gazeta.ru/politics/2019/11/07_a_12...,7 ноября в Белоруссии прошли выборы членов сов...,Вспомнить СССР: как Лукашенко провел выборы,В Белоруссии в день годовщины Октябрьской рево...,2019-11-07 19:55:08,В Белоруссии впервые прошли выборы членов сове...
3,https://www.gazeta.ru/culture/2020/03/01/a_129...,Народная артистка РСФСР Надежда Бабкина в инте...,«Он очень переживал»: Бабкина об отношениях с ...,Народная артистка РСФСР Надежда Бабкина в инте...,2020-03-01 16:50:06,"Надежда Бабкина рассказала, как ей удалось сбр..."
4,https://www.gazeta.ru/business/2020/02/06/1294...,Депутат Верховной рады от партии «Слуга народа...,«Поддерживают Россию»: почему Киев не платит п...,Украина не должна выплачивать пенсии жителям Д...,2020-02-06 12:41:24,В Верховной Раде раскритиковали законопроект о...
5,https://www.gazeta.ru/army/2019/11/13/12810158...,Журналист Питер Суппли Бенсон датского издания...,«Новый кулак в Арктике»: в Дании испугались «И...,Датского журналиста обеспокоила активность Рос...,2019-11-13 13:03:37,Журналист Питер Суппли Бенсон назвал «новым ку...
6,https://www.gazeta.ru/politics/2019/10/03_a_12...,Выступление главы российской делегации Петра Т...,"«Смотрите, что у вас происходит»: как прервали...",Активист сорвал выступление главы делегации РФ...,2019-10-03 20:46:08,На заседании Парламентской ассамблеи Совета Ев...
7,https://www.gazeta.ru/politics/2020/01/03_a_12...,Самые известные российские пранкеры Владимир «...,Именем Греты Тунберг: Вован и Лексус разыграли...,Пранкеры Вован и Лексус разыграли члена конгре...,2020-01-03 19:25:16,Российские пранкеры Владимир «Вован» Кузнецов ...
8,https://www.gazeta.ru/culture/2019/11/06/a_127...,Российский актер и театральный педагог Виталий...,Умер актер сериала «Улицы разбитых фонарей»,Актер сериала «Улицы разбитых фонарей» Виталий...,2019-11-06 14:17:33,Российский актер Виталий Жигалин ушел из жизни...
9,https://www.gazeta.ru/business/2020/01/23/1292...,Госдума одобрила в первом чтении внесенные пре...,Дождались: работающим пенсионерам восстановят ...,Работающие пенсионеры дождались индексации. Эт...,2020-01-23 19:32:12,Госдума одобрила в первом чтении поправки в Ко...


In [59]:
import razdel

def calc_lead_n_score(records, summary_col = 'summary', n=3, lower=True, nrows=1000):
    references = []
    predictions = []

    for i, record in records.iterrows():
        if i >= nrows:
            break

        summary = record[summary_col]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        text = text if not lower else text.lower()
        sentences = [sentence.text for sentence in razdel.sentenize(text)]
        prediction = " ".join(sentences[:n])
        predictions.append(prediction)

    calc_scores(references, predictions)

In [60]:
calc_lead_n_score(df_m, n=10)

Count: 10
Ref: работающие пенсионеры дождались индексации. это следует из уже одобренных в первом чтении поправок в конституцию. так что работающие пенсионеры смогут получить замороженную с 2016 года прибавку к пенсии. в россии их насчитывается более девяти миллионов. эксперты называют это восстановлением справедливости.
Hyp: госдума одобрила в первом чтении внесенные президентом рф владимиром путиным поправки в конституцию. в его основу легли предложения, которые президент россии владимир путин озвучил в послании федеральному собранию. одно из них — регулярная индексация, формирование системы пенсионного обеспечения граждан россии «на основе принципов всеобщности, справедливости и солидарности поколений». сопредседатель профильной рабочей группы по конституции россии талия хабриева пояснила, что поправка об индексации пенсий распространяется и на работающих пенсионеров. по словам хабриевой, само правило сформулировано без изъятий, и возможна только конкретизация порядка в федеральном 

In [61]:
calc_lead_n_score(df_m, summary_col = 'summary_pred', n=10)

Count: 10
Ref: госдума одобрила в первом чтении поправки в конституцию россии. она закрепила, что государство готово выполнять своие обязательства
Hyp: госдума одобрила в первом чтении внесенные президентом рф владимиром путиным поправки в конституцию. в его основу легли предложения, которые президент россии владимир путин озвучил в послании федеральному собранию. одно из них — регулярная индексация, формирование системы пенсионного обеспечения граждан россии «на основе принципов всеобщности, справедливости и солидарности поколений». сопредседатель профильной рабочей группы по конституции россии талия хабриева пояснила, что поправка об индексации пенсий распространяется и на работающих пенсионеров. по словам хабриевой, само правило сформулировано без изъятий, и возможна только конкретизация порядка в федеральном законе. о том, что из нынешних поправок в конституцию проистекает и индексация пенсий работающим пенсионерам, заявил и депутат госдумы олег шеин. по его словам, увеличение выпл

#### Вывод: Метрика на сгенерированных заголовках заметно хуже

#### 2.(дополнительно) Сделать генерацию заголовков для статьи (обучить модель для генерации заголовков)

Docs:
- https://huggingface.co/docs/transformers/main/en/notebooks
- https://github.com/huggingface/notebooks/blob/main/examples/summarization.ipynb

In [73]:
df_train = pd.DataFrame(train_records)
df_eval = pd.DataFrame(val_records)

In [80]:
model_checkpoint = "t5-small"

In [83]:
# !pip install rouge_score

In [84]:
from datasets import load_metric

metric = load_metric("rouge")

In [85]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [229]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

loading file https://huggingface.co/cointegrated/rut5-base-absum/resolve/main/spiece.model from cache at /Users/maximdoroshenko/.cache/huggingface/transformers/8a1eac3ebf60aa372c248f6a0c1746950e53dbe4854a6c7da4214a232b8d7ef3.b846524fbcbf3cf81e2302f8087043922ca4c445b4016bf16e707f7e2240a3e6
loading file https://huggingface.co/cointegrated/rut5-base-absum/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/cointegrated/rut5-base-absum/resolve/main/special_tokens_map.json from cache at /Users/maximdoroshenko/.cache/huggingface/transformers/0580647d92af8f2d54e8e5246c0f1d4a915857e12f8b5b329cbc2b10aecd5715.294ebaa4cd17bb284635004c92d2c4d522ec488c828dcce0c2471b6f28e3fe82
loading file https://huggingface.co/cointegrated/rut5-base-absum/resolve/main/tokenizer_config.json from cache at /Users/maximdoroshenko/.cache/huggingface/transformers/261c267054279f7a8a33af270db14f22624d21d277016abcb4019ef5e179b2f5.2dd03d206e8192151f6174bd84b39de784ed51f39735f40fdff39fa02be8

In [230]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"].to_list(), max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [231]:
# preprocess_function(df_train[:2])

In [232]:
tokenized_df_train = preprocess_function(df_train[:100])

In [233]:
tokenized_df_eval = preprocess_function(df_eval[:50])

In [234]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

loading configuration file https://huggingface.co/cointegrated/rut5-base-absum/resolve/main/config.json from cache at /Users/maximdoroshenko/.cache/huggingface/transformers/f1a153bb308fca4a2a02f4051ed80cf69897520df9b453f54367821e24a37a38.7f4839e0529489637e46ac6b0406b7d092cbda7090828c963dd6ffdd795da468
Model config T5Config {
  "_name_or_path": "/gd/MyDrive/models/rut5-base-absum",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5T

In [235]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
#     fp16=True,
#     push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [236]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [237]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [238]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_df_train,
    eval_dataset=tokenized_df_eval,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [239]:
trainer.train()

***** Running training *****
  Num examples = 3
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1


KeyError: 'Indexing with integers (to access backend Encoding for a given batch index) is not available when using Python based tokenizers'